In [1]:
import numpy as np
import pandas as pd
import re
from datetime import datetime

In [2]:
%run validator_cui.ipynb #importa functia validare_cui()
%run validator_cnp.ipynb #importa functia validare_cnp()
%run verificare_tva.ipynb #importa functia verificare_tva()

In [3]:
# path="C:\\Users\\zmeura\\Desktop\\rova\\10_Octombrie\\1_Cumparari.xlsx"
# mm=pd.read_excel(path, usecols="B:G",skiprows=range(0,3))
# mm.dropna(axis=0, how='all', inplace=True);
# mm['Numar document'] = mm['Numar document'].apply(lambda x: re.findall("\d+", str(x))[0])

In [27]:
path2="1_iesiri.xlsx"
mm2=pd.read_excel(path2, usecols="B:G",skiprows=range(0,3))
mm2.dropna(axis=0, how='all', inplace=True);
mm2['Numar document'] = mm2['Numar document'].apply(lambda x: int(re.findall("\d+", str(x))[0]))
mm2['CIF'] = mm2['CIF'].apply(lambda x: int(re.findall("\d+", str(x))[0]))
mm2['PJ_PF'] = ['PJ' if len(str(i))<10 else 'PF' for i in mm2['CIF']]
mm2['vcui'] = mm2['CIF'].apply(lambda x: validare_cui(int(x)))
mm2['vcnp'] = mm2['CIF'].apply(lambda x: validare_cnp(str(x)))

In [28]:
lista_data_facturi=[datetime.datetime.strptime(i,'%d.%m.%Y').strftime('%Y-%m-%d') for i in mm2['Data']]
nume_anaf,lista_tva = verificare_tva(mm2['CIF'].tolist(),lista_data_facturi, mm2['vcui'].tolist())

ANAF-Status Cerere TVA 200 OK


In [29]:
mm2['nume_anaf'] = nume_anaf
mm2['status_tva'] = lista_tva

In [30]:
probleme=[]
for index, val in enumerate(mm2['PJ_PF']):
    if val=='PJ':
        if mm2['vcui'][index]:
            probleme.append('OK')
        else:
            probleme.append('CUI Invalid')
    else:
        if mm2['vcnp'][index]:
            probleme.append('OK')
        else:
            probleme.append('CNP Invalid')

mm2['problem'] = probleme

In [31]:
#Verificari
print('Chestii gresite')
pb_pj=0
pb_pf=0
for index, val in enumerate(mm2['problem']):
    if val=='CUI Invalid':
        print('CUI Invalid pentru firma: ',mm2['Beneficiar'][index], ", Numar document: ",mm2['Numar document'][index])
        pb_pj+=1
    elif val=='CNP Invalid':
        print('CNP Invalid pentru persoana fizica: ',mm2['Beneficiar'][index],", Numar document: ",mm2['Numar document'][index])
        pb_pf+=1
print("Total probleme firme: ",pb_pj)
print("Total probleme persoane fizice: ",pb_pf)

Chestii gresite
Total probleme firme:  0
Total probleme persoane fizice:  0


In [32]:
from IPython.core.display import HTML
display(HTML(mm2.to_html()))

,Data,Numar document,Beneficiar,CIF,Total (include TVA),Numar chitanta,PJ_PF,vcui,vcnp,nume_anaf,status_tva,problem
0,03.12.2018,17117,CERIGO EXPRES SRL,35088944,30.000,12872.0,PJ,True,False,CERIGO EXPRES S.R.L.-D.,True,OK
1,03.12.2018,17118,PRELCHIM SRL,7059714,470.260,0.0,PJ,True,False,PRELCHIM SRL,True,OK
2,03.12.2018,17119,EUROPAN PROD SA,6833760,60.000,12873.0,PJ,True,False,EUROPAN PROD SA,True,OK
3,03.12.2018,17120,MATDAN SERVICE S.R.L.,6828267,249.810,0.0,PJ,True,False,MATDAN SERVICE SRL,True,OK
4,03.12.2018,17121,MARDAN EXPRESS SRL,35260410,130.220,0.0,PJ,True,False,MARDAN EXPRESS S.R.L.,True,OK
5,03.12.2018,17122,GENERAL AUTO INTERNATIONAL S.R.L.,12932270,300.000,0.0,PJ,True,False,GENERAL AUTO INTERNATIONAL SRL,True,OK
6,04.12.2018,17123,ADRAGOLA SRL,15868198,869.300,0.0,PJ,True,False,ADRAGOLA SRL,True,OK
7,04.12.2018,17124,MARDAN EXPRESS SRL,35260410,471.540,12874.0,PJ,True,False,MARDAN EXPRESS S.R.L.,True,OK
8,04.12.2018,17125,SIMOCAR SRL,15206875,60.000,12875.0,PJ,True,False,SIMOCAR SRL,True,OK
9,04.12.2018,17126,START-KING S.R.L.,13124957,300.000,0.0,PJ,True,False,START-KING SRL,True,OK
